# Caricamento dei modelli:

### 1) SAM2

In [1]:
import shutil
import matplotlib
from groundingdino.util.inference import load_model, load_image, predict, annotate

import numpy as np
import torchvision.transforms as transforms
#from sklearn.ensemble import IsolationForest
from torchvision.ops import box_convert
import os
from scipy.ndimage import generic_filter
import utility

# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import torchvision
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from sam2.build_sam import build_sam2, build_sam2_video_predictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator, SAM2ImagePredictor

In [2]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

def check_bfloat16_cublas_support():
    """Test if the GPU supports bfloat16 in cublasGemmStridedBatchedEx operations specifically"""
    if not torch.cuda.is_available():
        print("CUDA not available")
        return False

    try:
        # Create tensors that will specifically trigger cublasGemmStridedBatchedEx
        # Use larger dimensions and batch size to ensure the specific CUBLAS path is taken
        batch_size = 8
        seq_len = 64
        hidden_size = 128

        # Create example inputs similar to transformer attention operations
        query = torch.randn(batch_size, seq_len, hidden_size, device="cuda").to(torch.bfloat16)
        key = torch.randn(batch_size, seq_len, hidden_size, device="cuda").to(torch.bfloat16)
        value = torch.randn(batch_size, seq_len, hidden_size, device="cuda").to(torch.bfloat16)

        # This will trigger cublasGemmStridedBatchedEx internally
        # First compute query @ key.transpose(-2, -1) which uses batched matmul
        attn_weights = torch.bmm(query, key.transpose(1, 2))
        torch.cuda.synchronize()

        # Then compute attn_weights @ value which uses another batched matmul
        attn_output = torch.bmm(attn_weights.softmax(dim=-1), value)
        torch.cuda.synchronize()

        print("GPU fully supports bfloat16 in cublasGemmStridedBatchedEx")
        return True

    except RuntimeError as e:
        if "CUBLAS_STATUS_NOT_SUPPORTED" in str(e):
            print(f"bfloat16 not fully supported: {str(e)}")
            return False
        else:
            print(f"Other error occurred: {str(e)}")
            raise

if device.type == "cuda":
    if check_bfloat16_cublas_support():
        print("Using bfloat16 precision")  # use bfloat16 for the entire notebook
        torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    else:
        print("bfloat16 not supported, using float32 instead")  # older GPUs like Titan X do not support CUDNN ops with bfloat16...
        torch.autocast("cuda", dtype=torch.float32).__enter__()
    #torch.autocast("cuda", dtype=torch.float32).__enter__()
    # turn on tfloat32 for Ampere GPUs
    # (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

# loading SAM parameters
sam2_checkpoint = "./models/sam2.1/sam2.1_hiera_tiny.pt"
model_cfg_path = "./configs/sam2.1/sam2.1_hiera_t.yaml"


using device: cuda
bfloat16 not fully supported: CUDA error: CUBLAS_STATUS_NOT_SUPPORTED when calling `cublasGemmStridedBatchedEx(handle, opa, opb, (int)m, (int)n, (int)k, (void*)&falpha, a, CUDA_R_16BF, (int)lda, stridea, b, CUDA_R_16BF, (int)ldb, strideb, (void*)&fbeta, c, CUDA_R_16BF, (int)ldc, stridec, (int)num_batches, compute_type, CUBLAS_GEMM_DEFAULT_TENSOR_OP)`
bfloat16 not supported, using float32 instead


In [3]:
device = device
sam2 = build_sam2(model_cfg_path, sam2_checkpoint, device=device)
# mask_generator = SAM2AutomaticMaskGenerator(
#     sam2,
#     points_per_side=32,  # Increase for more detailed segmentation
#     pred_iou_thresh=0.7,  # Adjust threshold based on your needs 0.86 def
#     stability_score_thresh=0.92,
#     stability_score_offset=0.7,
#     crop_n_layers=1,
#     crop_n_points_downscale_factor=2
# )
image_predictor = SAM2ImagePredictor(sam2)
video_predictor = build_sam2_video_predictor(model_cfg_path, sam2_checkpoint, device=device)

### 2) Grounding DINO

In [4]:
model = load_model("./configs/groundingdino/GroundingDINO_SwinT_OGC.py", "./models/groundingdino/groundingdino_swint_ogc.pth",device)
#IMAGE_PATH = ""
TEXT_PROMPT = "object ."
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

final text_encoder_type: bert-base-uncased


## Utilizzo su Dataset 'FOSH'


In [ ]:
from utility import *
print(os.getcwd())  # This prints the current directory

# Where to retrive images
folder_path = "inserimento_oggetti"

# Extract frames
frame_names = [
    p for p in os.listdir(folder_path)
    if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
]
# Sorting frames
frame_names.sort(key=lambda p: int(os.path.splitext(p)[0].split('_')[-1]))

#setting the railway box
railway_box = np.array([0, 256, 640, 512])

# Setting statistics
gd_iou, h_iou, otsu_iou = 0, 0, 0
gd_dice , h_dice, otsu_dice = 0, 0, 0
gd_precision , h_precision, otsu_precision = 0, 0, 0
gd_recall, h_recall, otsu_recall = 0, 0, 0

n_frames = 0
skipped = 0

try:
    for p in frame_names:
        print('----------------------')
        image =  cv2.imread(os.path.join(folder_path, p))
        ground_truth = cv2.imread(utility.find_corresponding_segmentation(p, 'segmentazione_oggetti'), cv2.IMREAD_GRAYSCALE)

        if cv2.countNonZero(ground_truth) < 170:
            print("oggetto troppo piccolo")
            skipped += 1
            continue

        if n_frames % 20 == 0:
            print("frame saltati: ", skipped)

        name = str(p)
        print("Analyzing image:", name)
        print()
        image_predictor.set_image(image)

        object_mask_gd, m_h, m_l = None, None, None
        n_frames += 1

        ### GROUNDING DINO + SAM 2 METHOD


        # Finding objects in the bounding box with Grounding DINO
        image_source, gd_image = load_image(folder_path+'/'+name)

        gd_boxes, logits, phrases = predict(
            model=model,
            image=gd_image,
            caption=TEXT_PROMPT,
            box_threshold=BOX_TRESHOLD,
            text_threshold=TEXT_TRESHOLD,
            device=device,
        )

        h, w, _ = image_source.shape
        gd_boxes = gd_boxes * torch.Tensor([w, h, w, h])
        gd_boxes = box_convert(boxes=gd_boxes, in_fmt="cxcywh", out_fmt="xyxy").numpy()

        for box in gd_boxes:
            if utility.is_contained(box, railway_box,1):
                object_mask_gd, score_gd, _ = image_predictor.predict(
                    box=box,
                    multimask_output=False,
                )
                # salvataggio dell'immagine
                utility.show_masks(image, object_mask_gd, score_gd,borders=False,show=False,savefig=True,save_path="debug/gd2/",
                                   save_name=name,box_coords=box)


                statistics_map_gd = utility.segmentation_metrics(object_mask_gd, ground_truth)
                for metric in statistics_map_gd:
                    if metric == 'IoU':
                        gd_iou += statistics_map_gd[metric]
                    elif metric == 'Dice':
                        gd_dice += statistics_map_gd[metric]
                    elif metric == 'Precision':
                        gd_precision += statistics_map_gd[metric]
                    elif metric == 'Recall':
                        gd_recall += statistics_map_gd[metric]

        print("Grounding Dino - frame: ",n_frames, ", IoU: ", gd_iou/n_frames, ", Dice: ", gd_dice/n_frames,
              ", Precision: ", gd_precision/n_frames, ", Recall: ", gd_recall/n_frames)




        ### HOLES METHOD



        # Creating a grid of points for the railway's mask
        points ,labels = utility.create_grid(railway_box, points_per_row=[3, 4, 5])

        masks, scores, logits = image_predictor.predict(
            point_coords=points,
            point_labels=labels,
            #box=railway_box,
            multimask_output=False,
            return_logits=True,
        )
        sorted_ind = np.argsort(scores)[::-1]
        masks = masks[sorted_ind]
        scores = scores[sorted_ind]
        logits = logits[sorted_ind]

        # Displaying the image with the grid approach, use the parameters to show or save
        utility.show_masks(image,masks>0,scores,input_labels=labels,point_coords=points,show=False,savefig=True,
                           save_path='debug/grid/',save_name=p)

        l = np.squeeze(masks)
        l[l < 0] = 0

        sigmoid_mask = utility.sigmoid(l)  # Values are now in range [0, 1]

        #Convert to uint8 (0-255 range for OpenCV)
        sigmoid_mask_2d = (sigmoid_mask * 255).astype(np.uint8)  # Shape: (1, H, W)

        sig = (sigmoid_mask_2d > 127).astype(np.uint8) # binarify the mask to search for holes

        coord_holes, holes_labels = utility.find_holes(sig, 100)

        if len(coord_holes)>0:

            m_h,s_h,_ = image_predictor.predict(
                point_coords=coord_holes,
                point_labels=holes_labels,
                multimask_output=False,
            )
            utility.show_masks(image, m_h, s_h, borders=False, show=False, savefig=True, point_coords=coord_holes,
                               input_labels=holes_labels, save_path='debug/holes/', save_name=name)

            d_h = utility.segmentation_metrics(m_h, ground_truth)
            for metric in d_h:
                if metric == 'IoU':
                    h_iou += d_h[metric]
                elif metric == 'Dice':
                    h_dice += d_h[metric]
                elif metric == 'Precision':
                    h_precision += d_h[metric]
                elif metric == 'Recall':
                    h_recall += d_h[metric]

        print("Holes - frame: ", n_frames, ", IoU: ", h_iou / n_frames, ", Dice: ", h_dice / n_frames, ", Precision: ", h_precision / n_frames, ", Recall: ", h_recall / n_frames)



        ###  OTSU METHOD

        #Creating an high-pass filter upon railway mask logits using OTSU for the calculation of the threshold

        thresh_val, binary_mask = cv2.threshold(sigmoid_mask_2d, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        binary_mask[binary_mask > 127] = 1


        if binary_mask.dtype != np.uint8:
            binary_mask = binary_mask.astype(np.uint8)


        coord_holes, o_labels = utility.find_holes(binary_mask, 100)
        if len(coord_holes) > 0:
            m_o, s_o, _ = image_predictor.predict(
                point_coords=coord_holes,
                point_labels=o_labels,
                multimask_output=False,
            )
            utility.show_masks(image, m_o, np.array([0]), borders=False, point_coords=coord_holes, input_labels=o_labels,
                               show=False, savefig=True, save_path='debug/otzu/', save_name=name)

            d_o = utility.segmentation_metrics(m_o, ground_truth)
            for metric in d_o:
                if metric == 'IoU':
                    otsu_iou += d_o[metric]
                elif metric == 'Dice':
                    otsu_dice += d_o[metric]
                elif metric == 'Precision':
                    otsu_precision += d_o[metric]
                elif metric == 'Recall':
                    otsu_recall += d_o[metric]


        print("Otsu - frame: ", n_frames, ", IoU: ", otsu_iou / n_frames, ", Dice: ", otsu_dice / n_frames,
              ", Precision: ", otsu_precision / n_frames, ", Recall: ", otsu_recall / n_frames)
except(KeyboardInterrupt, SystemExit):
    print("Exiting...")
    print('iou', gd_iou, h_iou, otsu_iou)
    print('dice', gd_dice, h_dice, otsu_dice)
    print('precision', gd_precision, h_precision, otsu_precision)
    print('recall', gd_recall, h_recall, otsu_recall)
    print('n_frames ', n_frames)
    print('skipped ',skipped)

## Anomaly detection (Grounding DINO + SAM2)


In [5]:
#rd.video_analysis('s_c_f','seg_new_video', model)

video_dir = 'test_video/test_video_san_donato/'
segmented_video_dir = 'test_video/output'


# frame-based processing

In [ ]:
try:
    #Extract frames
    frame_names = [
        p for p in os.listdir(video_dir)
        if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
    ]
    #Sorting frames
    frame_names.sort(key=lambda p: int(os.path.splitext(p)[0].split('_')[-1]))
    #Setting the inference_state
    inference_state = video_predictor.init_state(video_path=video_dir)

    ann_frame_idx = 0  # the frame index we interact with
    ann_obj_id = 1  # give a unique id to each object we interact with (it can be any integers)

    # Finding the railway and objects bounding box with G DINO
    dino_boxes, phrases, dino_score = utility.grounding_Dino_analyzer(video_dir+'/'+frame_names[0],model,
                            'railway . object .', device)
    max_score_railway = 0
    railway_box = None
    object_points = []

    # FIXME we should check if those bounding box objects are inside the railway box, consider using utility.is_contained()
    for i, phrase in enumerate(phrases):
        if phrase == 'railway' and dino_score[i] > max_score_railway:
            railway_box = dino_boxes[i]
            max_score_railway = dino_score[i]
        if phrase == 'object':
            x_min, y_min, x_max, y_max = dino_boxes[i]
            x_center = (x_min + x_max) // 2
            y_center = (y_min + y_max) // 2
            object_points.append([x_center, y_center])

    # Setting the railway mask
    _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=ann_frame_idx,
        obj_id=ann_obj_id,
        box=railway_box,
    )
    id_objects = []

    # Setting the objects found in the first frame if any
    for obj_point in object_points:
        print('obj_point ', obj_point)
        ann_obj_id += 1
        id_objects.append(ann_obj_id)
        _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=ann_frame_idx,
            obj_id=ann_obj_id,
            points=[obj_point],
            labels= np.array([1], np.int32),
        )

    idx_frame = 0
    video_segments = {}  # video_segments contains the per-frame segmentation results

    while idx_frame < len(frame_names):
        last_masks = {}

        # Starting the inference propagation in future frames
        for out_frame_idx, out_obj_ids, out_mask_logits in video_predictor.propagate_in_video(inference_state, start_frame_idx=idx_frame):

            video_segments[out_frame_idx] = {
                out_obj_id: (out_mask_logits[i] > 0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }


            # plt.close('all')
            # plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
            #
            # for out_obj_id, out_mask in video_segments[out_frame_idx].items():
            #     utility.show_mask_v(out_mask, plt.gca(), obj_id=out_obj_id)
            #
            # plt.show()

            idx_frame += 1

            # If we're in the 15th frame we stop the propagation to check for newer objects
            if idx_frame % 15 == 0:
                for out_obj_id, out_mask in video_segments[out_frame_idx].items():
                    last_masks[out_obj_id] = out_mask
                break

        # Using Grounding DINO on the last frame
        obj_boxes, _, obj_scores= utility.grounding_Dino_analyzer(video_dir+'/'+frame_names[idx_frame-1], model,'object .',device)
        new_boxes = []

        #check if the objects found have already been seen and followed
        for obj_box in obj_boxes:
            contained = False
            for idx,mask in last_masks.items():
                #print('lll', mask.shape)
                if utility.is_mask_in_box(mask, obj_box):
                    #print("C'era di già")
                    contained = True
                    break
            if contained:
                continue
            else:
                new_boxes.append(obj_box)


        # saving those 15 frames in the segmented_video_dir
        for idx_frame_proc in range(idx_frame - 15, idx_frame):
            plt.close('all')
            plt.imshow(Image.open(os.path.join(video_dir, frame_names[idx_frame_proc])))

            for out_obj_id, out_mask in video_segments[idx_frame_proc].items():
                utility.show_mask_v(out_mask, plt.gca(), obj_id=out_obj_id)
            plt.savefig(os.path.join(segmented_video_dir, frame_names[idx_frame_proc]),
                        bbox_inches='tight', pad_inches=0)
            #plt.show()

        # If we find new objects we add them to the last frame using the same inference_state, then we can restart the  propagation process
        for obj_box in new_boxes:
            print("new box ", obj_box)
            ann_obj_id += 1

            x_min, y_min, x_max, y_max = obj_box
            x_center = (x_min + x_max) // 2
            y_center = (y_min + y_max) // 2

            _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx= idx_frame,
                obj_id=ann_obj_id,
                points=[[x_center, y_center]],
                labels=np.array([1], np.int32),
            )
        # segmented_img,_ = draw_mask_2(np.array(Image.open(os.path.join(video_dir, frame_names[out_frame_idx]))), out_mask_logits[0].cpu().numpy())
        # im = Image.fromarray(segmented_img)
        # save_path = os.path.join(segmented_video_dir, f"frame_{out_frame_idx:03d}.png")
        # im.save(save_path)# max_logit_value = out_mask_logits[0].max().item()


        # plt.imshow(out_mask_logits.squeeze(), cmap='viridis')
        # plt.colorbar(label='Valore dei Logits')
        # plt.title("Logits della Maschera")
        # plt.axis('off')
        # plt.show()
        # if out_frame_idx == 2:
        #     print("Annotiamo l'anomalia")
        #     # Let's add a negative click on this frame at (x, y) = (82, 415) to refine the segment
        #     pp = np.array([[550, 370]], dtype=np.float32)
        #     # for labels, `1` means positive click and `0` means negative click
        #     ll = np.array([0], np.int32)
        #     _, out_obj_ids, out_mask_logits = self.video_predictor.add_new_points_or_box(
        #         inference_state=inference_state,
        #         frame_idx=out_frame_idx,
        #         obj_id=ann_obj_id,
        #         points=pp,
        #         labels=ll,
        #     )
        # plt.figure(figsize=(9, 6))
        # plt.title(f"frame {out_frame_idx}")
        # plt.imshow(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
        # utility.show_mask_v((out_mask_logits[0] > threshold).cpu().numpy(), plt.gca(), obj_id=out_obj_ids[0])
        # if out_frame_idx == 100:
        #     plt.close('all')



    # vis_frame_stride = 1
    # plt.close("all")
    # for out_frame_idx in range(0, len(frame_names), vis_frame_stride):
    #     # image = np.asarray(Image.open(os.path.join(video_dir, frame_names[out_frame_idx])))
    #     for out_obj_id, out_mask in video_segments[out_frame_idx].items():
    #         utility.show_mask_v(out_mask, plt.gca(), obj_id=out_obj_id)
    #         # segmented_img = draw_mask(image, out_mask)
    #         # im = Image.fromarray(segmented_img)
    #         # save_path = os.path.join(segmented_video_dir, f"frame_{out_frame_idx:03d}.png")
    #         # im.save(save_path)
    #     plt.savefig(os.path.join(segmented_video_dir, frame_names[out_frame_idx]))
except(KeyboardInterrupt, SystemExit):
    print("Exiting...")

# Video-based processing

Refactored version to reduce CUDA memory usage and improve performance by processing frames in batches.
Avoids reprocessing frames from the beginning by using a stateful approach with SAM2.

In [7]:
import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import gc
import time

# Path configurations
video_path = "test_video/ir_2025-01-31_05-09-12.mp4"  # Can be camera index (0, 1) for webcam
segmented_video_dir = "test_video/output"

BOX_TRESHOLD = 0.22  # Threshold for Grounding DINO box detection
TEXT_TRESHOLD = 0.18  # Threshold for Grounding DINO text detection

# Create output directory if it doesn't exist
os.makedirs(segmented_video_dir, exist_ok=True)

# Create a temporary directory for frame storage
temp_dir = os.path.join("temp_frames")
os.makedirs(temp_dir, exist_ok=True)

# Open the video stream
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception(f"Could not open video stream: {video_path}")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video stream at {fps} FPS, resolution: {width}x{height}")

# Object tracking variables
ann_obj_id = 1  # Object ID counter
last_masks = {}  # Store the last known mask for each object
frame_idx = 0
railway_box = None  # Store railway box for future reference

try:
    while True:
        print(f"\n--- Processing frame {frame_idx} ---")
        start_time = time.time()

        # Read the next frame
        success, frame = cap.read()
        if not success:
            print("End of stream or error reading frame")
            break

        # Clear temp directory
        for file in os.listdir(temp_dir):
            os.remove(os.path.join(temp_dir, file))

        # Save current frame to temp directory
        frame_path = os.path.join(temp_dir, "000000.jpg")
        cv2.imwrite(frame_path, frame)

        # Convert to RGB for visualization
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Initialize new state for this frame
        torch.cuda.empty_cache()
        gc.collect()
        inference_state = video_predictor.init_state(video_path=temp_dir)

        # Process based on frame index
        if frame_idx == 0:
            # First frame: detect railway and objects with Grounding DINO
            dino_boxes, phrases, dino_scores = utility.grounding_Dino_analyzer(
                frame_path, model, 'railway . object .', device, BOX_TRESHOLD=BOX_TRESHOLD, TEXT_TRESHOLD=TEXT_TRESHOLD
            )

            # Find railway box and object points
            max_score_railway = 0
            object_points = []

            for i, phrase in enumerate(phrases):
                if phrase == 'railway' and dino_scores[i] > max_score_railway:
                    railway_box = dino_boxes[i]
                    max_score_railway = dino_scores[i]
                elif phrase == 'object':
                    x_min, y_min, x_max, y_max = dino_boxes[i]
                    object_points.append([(x_min + x_max) // 2, (y_min + y_max) // 2])

            print(f"Found railway: {railway_box is not None}, objects: {len(object_points)}")

            # Add railway to tracking
            if railway_box is not None:
                _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
                    inference_state=inference_state,
                    frame_idx=0,
                    obj_id=ann_obj_id,
                    box=railway_box,
                )

                # Store railway mask
                last_masks[ann_obj_id] = (out_mask_logits[0] > 0).cpu().numpy()

            # Add detected objects to tracking
            for obj_point in object_points:
                ann_obj_id += 1
                _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
                    inference_state=inference_state,
                    frame_idx=0,
                    obj_id=ann_obj_id,
                    points=[obj_point],
                    labels=np.array([1], np.int32),
                )

                # Store object mask
                idx = list(out_obj_ids).index(ann_obj_id) if ann_obj_id in out_obj_ids else 0
                last_masks[ann_obj_id] = (out_mask_logits[idx] > 0).cpu().numpy()
        else:
            # For non-first frames, transfer objects from previous frame
            for obj_id, mask in last_masks.items():
                # Convert mask to proper format and find center point
                mask_array = np.asarray(mask)
                if mask_array.ndim > 2:
                    mask_array = mask_array.squeeze()
                    if mask_array.ndim > 2:
                        mask_array = mask_array[0]

                # Find non-zero coordinates (points inside the mask)
                y_indices, x_indices = np.where(mask_array > 0)

                if len(y_indices) > 0:
                    # Use center of mass as representative point
                    center_y = int(np.mean(y_indices))
                    center_x = int(np.mean(x_indices))

                    # Special handling for railway (can use box instead of point)
                    if obj_id == 1 and railway_box is not None:
                        _, _, _ = video_predictor.add_new_points_or_box(
                            inference_state=inference_state,
                            frame_idx=0,
                            obj_id=obj_id,
                            box=railway_box,
                        )
                    else:
                        # Add object using its center point
                        _, _, _ = video_predictor.add_new_points_or_box(
                            inference_state=inference_state,
                            frame_idx=0,
                            obj_id=obj_id,
                            points=[[center_x, center_y]],
                            labels=np.array([1], np.int32),
                        )

        # Propagate all objects in current frame
        result = next(video_predictor.propagate_in_video(
            inference_state,
            start_frame_idx=0
        ))
        _, out_obj_ids, out_mask_logits = result

        # Update all masks for next frame
        for i, obj_id in enumerate(out_obj_ids):
            last_masks[obj_id] = (out_mask_logits[i] > 0).cpu().numpy()

        # Check for new objects periodically
        if frame_idx % 15 == 0 and frame_idx > 0:
            dino_boxes, phrases, _ = utility.grounding_Dino_analyzer(
                frame_path, model, 'object .', device, BOX_TRESHOLD=BOX_TRESHOLD, TEXT_TRESHOLD=TEXT_TRESHOLD
            )

            # Check each detected object
            for i, phrase in enumerate(phrases):
                if phrase == 'object':
                    x_min, y_min, x_max, y_max = dino_boxes[i]
                    center_x = (x_min + x_max) // 2
                    center_y = (y_min + y_max) // 2

                    # Check if this object is already tracked
                    already_tracked = False
                    for mask in last_masks.values():
                        mask_array = np.asarray(mask)
                        if mask_array.ndim > 2:
                            mask_array = mask_array.squeeze()
                            if mask_array.ndim > 2:
                                mask_array = mask_array[0]

                        # Check if point is inside any existing mask
                        if (0 <= int(center_y) < mask_array.shape[0] and
                          0 <= int(center_x) < mask_array.shape[1] and
                            mask_array[int(center_y), int(center_x)]):
                            already_tracked = True
                            break

                    # Add new object if not already tracked
                    if not already_tracked:
                        ann_obj_id += 1
                        print(f"New object {ann_obj_id} detected at frame {frame_idx}")

                        _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
                            inference_state=inference_state,
                            frame_idx=0,
                            obj_id=ann_obj_id,
                            points=[[center_x, center_y]],
                            labels=np.array([1], np.int32),
                        )

                        # Re-propagate with the new object
                        result = next(video_predictor.propagate_in_video(
                            inference_state,
                            start_frame_idx=0
                        ))
                        _, out_obj_ids, out_mask_logits = result

                        # Update masks dictionary
                        for i, obj_id in enumerate(out_obj_ids):
                            last_masks[obj_id] = (out_mask_logits[i] > 0).cpu().numpy()

        # Create visualization
        plt.figure(figsize=(8, 6))
        plt.imshow(frame_rgb)
        for obj_id, mask in last_masks.items():
            utility.show_mask_v(mask, plt.gca(), obj_id=obj_id)
        plt.savefig(os.path.join(segmented_video_dir, f"frame_{frame_idx:06d}.jpg"))
        plt.close()

        # Calculate processing time
        processing_time = time.time() - start_time
        print(f"Frame processed in {processing_time:.2f}s")

        # Increment frame counter
        frame_idx += 1

        # Clear memory for next iteration
        del inference_state
        gc.collect()
        torch.cuda.empty_cache()

except (KeyboardInterrupt, SystemExit):
    print("Exiting gracefully...")
finally:
    # Release resources
    cap.release()

    # Clean up temp directory
    for file in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, file))
    os.rmdir(temp_dir)

    print(f"Processing completed or interrupted after {frame_idx} frames")

Video stream at 25.0 FPS, resolution: 640x512

--- Processing frame 0 ---


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 29.04it/s]


Analysis with Grounding Dino


['railway', 'object', 'object'] tensor([0.7364, 0.5972, 0.3127])
Found railway: True, objects: 2


propagate in video:   0%|          | 0/1 [00:00<?, ?it/s]


...

['object', 'object', 'object'] tensor([0.5357, 0.4033, 0.2727])
Frame processed in 2.67s

--- Processing frame 751 ---


propagate in video:   0%|          | 0/1 [00:00<?, ?it/s]


Frame processed in 1.97s

--- Processing frame 752 ---


frame loading (JPEG): 100%|██████████| 1/1 [00:00<00:00, 25.92it/s]


Exiting gracefully...
Processing completed or interrupted after 752 frames


# TEST
Questa versione processa i frame in due thread: uno legge i dati e l'altro esegue l'inferenza. Ci sono ancora bug e può bloccarsi !

In [ ]:
import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import gc
import time
import threading
import queue
from concurrent.futures import ThreadPoolExecutor

# Path configurations
video_path = "test_video/test_video_san_donato.mp4"
segmented_video_dir = "test_video/output"

# Create output directory if it doesn't exist
os.makedirs(segmented_video_dir, exist_ok=True)

# Create a temporary directory for frame storage
temp_dir = os.path.join("temp_frames")
os.makedirs(temp_dir, exist_ok=True)

# Open the video stream
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise Exception(f"Could not open video stream: {video_path}")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Video stream at {fps} FPS, resolution: {width}x{height}")

# Create processing queues
frame_queue = queue.Queue(maxsize=2)  # Queue for frames waiting to be processed
result_queue = queue.Queue(maxsize=2)  # Queue for processed results waiting to be saved

# Threading lock for GPU operations
gpu_lock = threading.Lock()

# Object tracking variables
ann_obj_id = 1
last_masks = {}
railway_box = None
frame_idx = 0

# Function to read frames in a separate thread
def read_frames():
    global frame_idx
    while True:
        success, frame = cap.read()
        if not success:
            # Put None as a sentinel value to signal end of stream
            frame_queue.put(None)
            break

        # Convert frame to RGB for visualization
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Save current frame to temp directory
        frame_path = os.path.join(temp_dir, "000000.jpg")
        cv2.imwrite(frame_path, frame)

        # Put frame data in queue
        frame_data = {
            'index': frame_idx,
            'frame': frame,
            'frame_rgb': frame_rgb,
            'frame_path': frame_path
        }
        frame_queue.put(frame_data)
        frame_idx += 1

# Function to process frames using GPU in a separate thread
def process_frames():
    global ann_obj_id, last_masks, railway_box

    while True:
        # Get next frame from queue
        frame_data = frame_queue.get()

        # Check for end of stream
        if frame_data is None:
            result_queue.put(None)
            break

        idx = frame_data['index']
        frame = frame_data['frame']
        frame_rgb = frame_data['frame_rgb']
        frame_path = frame_data['frame_path']

        print(f"\n--- Processing frame {idx} ---")
        start_time = time.time()

        # Acquire GPU lock for model inference
        with gpu_lock:
            # Clear CUDA memory before processing
            torch.cuda.empty_cache()
            gc.collect()

            # Initialize inference state
            inference_state = video_predictor.init_state(video_path=temp_dir)

            # Process based on frame index
            if idx == 0:
                # First frame: detect railway and objects
                dino_boxes, phrases, dino_scores = utility.grounding_Dino_analyzer_plt(
                    frame_path, model, 'railway . object .', device
                )

                # Find railway box and object points
                max_score_railway = 0
                object_points = []

                for i, phrase in enumerate(phrases):
                    if phrase == 'railway' and dino_scores[i] > max_score_railway:
                        railway_box = dino_boxes[i]
                        max_score_railway = dino_scores[i]
                    elif phrase == 'object':
                        x_min, y_min, x_max, y_max = dino_boxes[i]
                        object_points.append([(x_min + x_max) // 2, (y_min + y_max) // 2])

                # Add railway to tracking
                if railway_box is not None:
                    _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
                        inference_state=inference_state,
                        frame_idx=0,
                        obj_id=ann_obj_id,
                        box=railway_box,
                    )

                    last_masks[ann_obj_id] = (out_mask_logits[0] > 0).cpu().numpy()

                # Add detected objects to tracking
                for obj_point in object_points:
                    ann_obj_id += 1
                    _, out_obj_ids, out_mask_logits = video_predictor.add_new_points_or_box(
                        inference_state=inference_state,
                        frame_idx=0,
                        obj_id=ann_obj_id,
                        points=[obj_point],
                        labels=np.array([1], np.int32),
                    )

                    if ann_obj_id in out_obj_ids:
                        idx = list(out_obj_ids).index(ann_obj_id)
                        last_masks[ann_obj_id] = (out_mask_logits[idx] > 0).cpu().numpy()
            else:
                # For non-first frames, transfer objects from previous frame
                for obj_id, mask in last_masks.items():
                    # Convert mask to proper format
                    mask_array = np.asarray(mask)
                    if mask_array.ndim > 2:
                        mask_array = mask_array.squeeze()
                        if mask_array.ndim > 2:
                            mask_array = mask_array[0]

                    # Find non-zero coordinates
                    y_indices, x_indices = np.where(mask_array > 0)

                    if len(y_indices) > 0:
                        # Calculate center of mass
                        center_y = int(np.mean(y_indices))
                        center_x = int(np.mean(x_indices))

                        # Add object to current frame
                        if obj_id == 1 and railway_box is not None:
                            video_predictor.add_new_points_or_box(
                                inference_state=inference_state,
                                frame_idx=0,
                                obj_id=obj_id,
                                box=railway_box,
                            )
                        else:
                            video_predictor.add_new_points_or_box(
                                inference_state=inference_state,
                                frame_idx=0,
                                obj_id=obj_id,
                                points=[[center_x, center_y]],
                                labels=np.array([1], np.int32),
                            )

            # Propagate masks in current frame
            result = next(video_predictor.propagate_in_video(
                inference_state,
                start_frame_idx=0
            ))
            _, out_obj_ids, out_mask_logits = result

            # Update masks for next frame
            frame_masks = {}
            for i, obj_id in enumerate(out_obj_ids):
                frame_masks[obj_id] = (out_mask_logits[i] > 0).cpu().numpy()
            last_masks = frame_masks

            # Check for new objects periodically
            if idx % 15 == 0 and idx > 0:
                dino_boxes, phrases, _ = utility.grounding_Dino_analyzer_plt(
                    frame_path, model, 'object .', device
                )

                # Check each detected object
                for i, phrase in enumerate(phrases):
                    if phrase == 'object':
                        x_min, y_min, x_max, y_max = dino_boxes[i]
                        center_x = (x_min + x_max) // 2
                        center_y = (y_min + y_max) // 2

                        # Check if already tracked
                        already_tracked = False
                        for mask in last_masks.values():
                            mask_array = np.asarray(mask)
                            if mask_array.ndim > 2:
                                mask_array = mask_array.squeeze()
                                if mask_array.ndim > 2:
                                    mask_array = mask_array[0]

                            if (0 <= int(center_y) < mask_array.shape[0] and
                                0 <= int(center_x) < mask_array.shape[1] and
                                mask_array[int(center_y), int(center_x)]):
                                already_tracked = True
                                break

                        # Add new object if needed
                        if not already_tracked:
                            ann_obj_id += 1
                            print(f"New object {ann_obj_id} detected at frame {idx}")

                            video_predictor.add_new_points_or_box(
                                inference_state=inference_state,
                                frame_idx=0,
                                obj_id=ann_obj_id,
                                points=[[center_x, center_y]],
                                labels=np.array([1], np.int32),
                            )

                            # Re-propagate with new object
                            result = next(video_predictor.propagate_in_video(
                                inference_state,
                                start_frame_idx=0
                            ))
                            _, out_obj_ids, out_mask_logits = result

                            # Update masks
                            for i, obj_id in enumerate(out_obj_ids):
                                last_masks[obj_id] = (out_mask_logits[i] > 0).cpu().numpy()

        # Calculate processing time
        processing_time = time.time() - start_time
        print(f"Frame {idx} processed in {processing_time:.2f}s")

        # Put results in output queue
        result_data = {
            'index': idx,
            'frame_rgb': frame_rgb,
            'masks': dict(last_masks)  # Make a copy of the masks dict
        }
        result_queue.put(result_data)

        # Clean up
        del inference_state
        gc.collect()

# Function to save visualization results in a separate thread
def save_results():
    while True:
        # Get processed result from queue
        result = result_queue.get()

        # Check for end of processing
        if result is None:
            break

        idx = result['index']
        frame_rgb = result['frame_rgb']
        masks = result['masks']

        # Create visualization
        plt.figure(figsize=(8, 6))
        plt.imshow(frame_rgb)
        for obj_id, mask in masks.items():
            utility.show_mask_v(mask, plt.gca(), obj_id=obj_id)
        plt.savefig(os.path.join(segmented_video_dir, f"frame_{idx:06d}.jpg"))
        plt.close()

        print(f"Frame {idx} visualization saved")

try:
    # Start threads for pipelined processing
    with ThreadPoolExecutor(max_workers=3) as executor:
        reader_future = executor.submit(read_frames)
        processor_future = executor.submit(process_frames)
        writer_future = executor.submit(save_results)

        # Wait for all threads to complete
        reader_future.result()
        processor_future.result()
        writer_future.result()

except (KeyboardInterrupt, SystemExit):
    print("Exiting gracefully...")
finally:
    # Release resources
    cap.release()

    # Clean up temp directory
    for file in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, file))
    os.rmdir(temp_dir)

    print(f"Processing completed or interrupted after {frame_idx} frame")